## 주의

이 tutorials 는 큰 용량의 메모리를 요구합니다. 충분한 메모리가 확보된 각자의 환경에서만 테스트 해보시길 권장합니다. 

Scikit-learn 의 logistic regression 을 이용하기 위하여 데이터를 한 번에 메모리에 올리기 때문인데, 대략 6 GB 를 이용합니다.

Scikit-learn 에서도 partial fit 함수를 이용하여 mini batch style 을 구현할 수 있습니다. Scikit-learn 은 근사 해법들이 잘 구현되어 있습니다. 혹은 PyTorch 나 TensorFlow 를 이용하여 네트워크를 구성하셔도 좋습니다. 이 때는 근사 해법들은 따로 구현하셔야 합니다.

## Load dataset

Comments 는 list of str 형식의 문장을 yield 하는 데이터클래스 입니다. 영화평 데이터는 (영화 id, 영화평, 평점) 의 column data 이기 때문에 영화평만 선택하여 띄어쓰기 기준으로 split 합니다.

In [1]:
import config
from navermovie_comments import get_movie_comments_path

class Comments:
    def __init__(self, path):
        self.path = path

    def __iter__(self):
        def normalize(text):
            if text[:3] == '관람객':
                return text[3:].strip()
            return text

        with open(self.path, encoding='utf-8') as f:
            for i, doc in enumerate(f):
                idx, text, rate = doc.split('\t')
                yield normalize(text).split()

path = get_movie_comments_path(large=True, tokenize='soynlp_unsup')
sents = [sent for sent in Comments(path)]

for i, sent in enumerate(sents):
    if i >= 3:
        break
    print(sent)

soynlp=0.0.492
added lovit_textmining_dataset
['명불허전']
['왠지', '고사', '피의', '중간', '고사', '보다', '재미', '가', '없을듯', '해요', '만약', '보게', '된다면', '실망', '할듯']
['티아라', '사랑', '해', 'ㅜ']


## Scan vocabulary

데이터에 존재하는 단어들을 `min_count` 기준으로 필터링합니다. Return 값으로 vocabulary index 와 count 를 return 합니다.

In [2]:
from collections import defaultdict

def scan_vocabulary(sents, min_count, verbose=False):
    counter = defaultdict(int)
    for i, sent in enumerate(sents):
        if verbose and i % 100000 == 0:
            print('\rscanning vocabulary .. from %d sents' % i, end='')
        for word in sent:
            counter[word] += 1
    counter = {word:count for word, count in counter.items()
               if count >= min_count}
    idx_to_vocab = [vocab for vocab in sorted(counter,
                    key=lambda x:-counter[x])]
    vocab_to_idx = {vocab:idx for idx, vocab in enumerate(idx_to_vocab)}
    idx_to_count = [counter[vocab] for vocab in idx_to_vocab]
    if verbose:
        print('\rscanning vocabulary was done. %d terms from %d sents' % (len(idx_to_vocab), i+1))
    return vocab_to_idx, idx_to_vocab, idx_to_count

vocab_to_idx, idx_to_vocab, idx_to_count = scan_vocabulary(
    sents, min_count=10, verbose=True)

scanning vocabulary was done. 69541 terms from 3280685 sents


scan_vocabulary 의 return 값을 살펴봅니다.

In [3]:
print(idx_to_vocab[:5])
print(idx_to_count[:5])

['영화', '이', '관람', '객', '의']
[1128809, 866305, 600351, 526070, 489950]


## Word2Vec 을 이용한 seed set 만들기

미리 학습해둔 Word2Vec model 을 로딩합니다. 우리는 사람 이름을 인식하는 named entity recognizer 를 만들겁니다. `송강호`와 `디카프리오`의 Word2Vec 유사어는 사람 이름임을 알 수 있습니다. 각각 100 개씩의 유사어를 선택하여 이의 합집합을 seed_words 로 선택합니다. 총 172 개의 단어가 seeds 로 선택되었습니다.

In [4]:
from pprint import pprint
from navermovie_comments import load_trained_embedding


word2vec = load_trained_embedding(tokenize='soynlp_unsup')

pprint(word2vec.wv.most_similar('송강호', topn=10))
pprint(word2vec.wv.most_similar('디카프리오', topn=10))

seed_words = {word for word, _ in word2vec.most_similar('송강호', topn=100)}
seed_words.update({word for word, _ in word2vec.most_similar('디카프리오', topn=100)})

len(seed_words)

[('하정우', 0.9089709520339966),
 ('한석규', 0.882610023021698),
 ('오달수', 0.8565835356712341),
 ('이정재', 0.8554919362068176),
 ('김명민', 0.8462209105491638),
 ('이범수', 0.8421300649642944),
 ('설경구', 0.8421212434768677),
 ('황정민', 0.8380463719367981),
 ('손현주', 0.8376869559288025),
 ('김윤석', 0.8338450193405151)]
[('레오', 0.8400799036026001),
 ('톰하디', 0.8309459686279297),
 ('앤해서웨이', 0.775414764881134),
 ('앤헤서웨이', 0.7646037340164185),
 ('브래드피트', 0.7506041526794434),
 ('로다주', 0.7499849796295166),
 ('로버트드니로', 0.7493538856506348),
 ('콜린퍼스', 0.7378235459327698),
 ('히스레저', 0.7330209612846375),
 ('윌스미스', 0.7309495806694031)]


/home/lovit/anaconda3/envs/pytorch/lib/python3.7/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
/home/lovit/anaconda3/envs/pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  # Remove the CWD from sys.path while we load stuff.
/home/lovit/anaconda3/envs/pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  # This is added back by InteractiveShellApp.init_path()


172

## Features

우리는 window = 2 를 이용하여 한 단어 X[0] 의 앞, 뒤 각각 두 개의 단어 (총 4개의 단어)를 X[0] 의 features 로 이용합니다. 예를 들어 (a, b, c, d, e) 라는 단어가 등장하였고, X[0]=c 라면 X[-2]=a, X[-1]=b, X[1]=d, X[2]=e 입니다.

그리고 scan vocabulary 함수를 통하여 학습된 단어가 총 5 개라면 이들의 위치를 보존하면 X[0] 에 대한 feature space 를 20 만 차원으로 만들 수 있습니다. 만약 각 단어의 index 가 {a:0, b:1, c:2, d:3, e:4} 라면 X[0]=c 는 [0, 5+1, 10+3, 15+4] 를 features 로 가진다 표현할 수 있습니다.

feature_to_idx 는 이를 만드는 함수입니다. 문장 내에서 현재 단어 X[0] 의 위치를 i, 현재 단어 앞, 뒤 단어인 X[-1] 이나 X[1] 의 위치를 j, j 위치의 단어의 index 를 vocab_idx 라 할 때, 이 값의 feature index 를 출력합니다.

In [5]:
def feature_to_idx(i, j, vocab_idx, window, n_terms):
    if j < i:
        return n_terms * (j - i + window) + vocab_idx
    else:
        return n_terms * (j - i + window - 1) + vocab_idx

feature_to_idx(i=2, j=3, vocab_idx=3, window=2, n_terms=5)

13

idx_to_feature 는 반대로 feature index 를 feature 로 decode 합니다. feature idx 를 vocabulary 의 개수로 나눈 몫은 상대적 위치값이 되고, 나머지는 vocabulary idx 입니다.

In [6]:
def idx_to_feature(feature_idx, idx_to_vocab, window):
    # 몫
    position = feature_idx // len(idx_to_vocab)
    if position < window:
        feature = 'X[-%d] = ' % (window - position)
    else:
        feature = 'X[%d] = ' % (position - window + 1)
    # 나머지
    vocab_idx = feature_idx % len(idx_to_vocab)
    feature += idx_to_vocab[vocab_idx]
    return feature

idx_to_feature(13, 'a b c d e'.split(), window=2)

'X[1] = d'

이를 이용하여 학습데이터로부터 window classification 용 데이터를 만듭니다. X[0] 을 기준으로 X[-2], X[-1], X[1], X[2] 의 co-occurrence 를 계산하는 matrix 를 만듭니다. Sparse matrix 형식이기 때문에 rows, columns 를 따로 모읍니다. words 는 각 rows 에 해당하는 단어를 넣어둡니다.

아래 부분에서 scan vocabulary 의 결과에 포함되지 않은 단어는 건너 띄며, context words 의 범위는 문장의 맨 앞에서 문장의 맨 뒷 단어가 되도록 index 의 범위를 확인합니다.

```python
for i, word in enumerate(sent):
    if not (word in vocab_to_idx):
        continue

    b = max(0, i - window)
    e = min(i + window, n_words)
```

아래 구문을 통하여 sent[j] 의 단어 역시 scan vocabulary 의 결과에 포함되지 않으면 이를 이용하지 않습니다.

```python
for j in range(b, e):
    if i == j:
        continue
    j_idx = vocab_to_idx.get(sent[j], -1)
    if j_idx == -1:
        continue
```

In [7]:
import numpy as np
from scipy.sparse import csr_matrix

def create_dataset(vocab_to_idx, sentences, seed_words, window=2, test_data=False, verbose=True):

    def contain_seed(sent):
        for word in sent:
            if word in seed_words:
                return True
        return False

    n_terms = len(vocab_to_idx)

    rows = []
    cols = []
    words = []

    row_idx = 0
    col_idx = window * 2 * n_terms

    for i_sent, sent in enumerate(sentences):

        if verbose and i_sent % 10000 == 0:
            print('\rcreating train dataset {} rows from {} sents'.format(row_idx, i_sent), end='')

        if (not test_data) and (not contain_seed(sent)):
            continue

        n_words = len(sent)

        for i, word in enumerate(sent):
            if not (word in vocab_to_idx):
                continue

            b = max(0, i - window)
            e = min(i + window + 1, n_words)

            features = []
            for j in range(b, e):
                if i == j:
                    continue
                j_idx = vocab_to_idx.get(sent[j], -1)
                if j_idx == -1:
                    continue
                features.append(feature_to_idx(i, j, j_idx, window, n_terms))

            if not features:
                continue

            # sparse matrix element
            for col in features:
                rows.append(row_idx)
                cols.append(col)

            # words element
            words.append(word)

            row_idx += 1

    if verbose:
        print('\rtrain dataset {} rows from {} sents was created    '.format(row_idx, i_sent))

    # to csr matrix
    rows = np.asarray(rows, dtype=np.int)
    cols = np.asarray(cols, dtype=np.int)
    data = np.ones(rows.shape[0], dtype=np.int)
    X = csr_matrix((data, (rows, cols)), shape=(row_idx, col_idx))

    return X, words

In [8]:
X_train, words = create_dataset(vocab_to_idx, sents, seed_words, window=2, test_data=False)

train dataset 5217933 rows from 3280684 sents was created    


## Word2Vec 유사어를 이용하여 label vector 만들기

앞서 만든 X 의 row 에 해당하는 단어가 seed_words 에 포함될 경우, 이 rows 의 값을 1 로, 그렇지 않은 경우 0 으로 지정합니다.

172 개의 단어가 361,394 번 등장하였습니다.

In [9]:
y_train = np.zeros(X_train.shape[0], dtype=np.int)
for i, word in enumerate(words):
    if word in seed_words:
        y_train[i] = 1

y_train.sum()

361749

## Logistic Regression 을 이용한 window classifier 만들기

Logistic regression 을 학습합니다. seed words 를 positive class 로 예측하는 모델을 만듭니다. 그런데 실제로는 사람 이름이면서도 label 을 0 으로 가지는 데이터도 존재합니다. Logistic regression 은 이들에 대해서는 큰 확률값을 지닐 가능성이 높습니다. Training error 를 named entity 의 힌트로 이용하는 것입니다. 이 때의 training error 는 우리가 seed words 를 이용하여 엉성하게 데이터를 준비했기 때문에 발생하는 error 이기 때문입니다.

In [10]:
from sklearn.linear_model import LogisticRegression

logistic = LogisticRegression()
logistic.fit(X_train, y_train)

/home/lovit/anaconda3/envs/pytorch/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

학습에 이용한 X_train 은 크기가 그렇게 큰 편은 아니지만, 모든 문장을 prediction 을 위한 sparse matrix 로 변환하면 그 크기는 상당히 큽니다. 약 20 GB 의 메모리가 필요했습니다. 이런 경우에는 `batch_sents_size` 씩 테스트 문장을 잘라내어 데이터를 만든 뒤, prediction 결과만 따로 저장합니다. 일종의 minibatch 형식입니다.

다음 `day7_3_feedforward_window_classification_ner` 에서는 학습 과정도 Scikit-learn 을 이용한 minibatch 형식으로 구현해 봅니다.

In [11]:
import math

def minibatch_predict(sents, seed_words, model, batch_sents_size=100000):

    # prepare
    y_prob = []
    y_words = []
    n_sents = len(sents)
    n_batchs = math.ceil(n_sents / batch_sents_size)

    # for each batch
    for batch in range(n_batchs):

        # begin, end index
        b = batch * batch_sents_size
        e = min((batch + 1) * batch_sents_size, n_sents)

        # slicing list & create dataset
        batch_sents = sents[b:e]
        X, words = create_dataset(vocab_to_idx, batch_sents, seed_words, test_data=True, verbose=False)

        # append prediction probability & label word
        y_prob.append(model.predict_proba(X))
        y_words += words

        # verbose
        print('\rbatch prediction {} / {}'.format(batch+1, n_batchs), end='')
    print('\rbatch prediction {0} / {0}'.format(n_batchs))

    # merging stacked numpy.ndarray
    y_prob = np.vstack(y_prob)[:,1]
    # transform list of str to numpy.ndarray of str
    y_words=  np.asarray(y_words)

    return y_prob, y_words

y_prob, y_words = minibatch_predict(sents, seed_words, logistic)

batch prediction 33 / 33


총 43,000,283 개의 prediction 을 하였습니다.

In [12]:
y_prob.shape, y_words.shape

((43000283,), (43000283,))

Softmax regression 은 일단 학습데이터로부터 패턴을 학습하기 때문에 positive class 에는 큰 확률을, negative class 에는 작은 확률을 부여합니다. 하지만 여기에는 실제로 positive class 이지만, 우리가 label 을 negative 로 준 경우도 있습니다. Positive 의 개수가 negative 보다 작기 때문에 모델은 positive class 를 더 잘 맞추는 쪽으로 학습을 합니다 (Scikit-learn 은 loss 의 class weight 를 조정하는게 기본값입니다). 그렇기 때문에 실제로 positive 이나 negative 로 label 된 경우에는 training error 가 크게 났을 것입니다.

그리고 그 영향력이 적다 하더라도 실제로 positive 인데 negative 로 labeling 된 데이터에 의하여 positive 에 해당하는 데이터들의 확률도 1 보다는 작은 값에 가까워집니다. 그렇기 때문에 우리는 여유롭게 positive class 의 확률값이 0.3 보다 큰 경우들을 세어봅니다.

예를 들어 배우 `백윤식` 은 seed words 에 포함되지 않았지만 총 100 번 등장하였고, 그 중 95 번을 0.05 보다 큰 prediction probability 를 받았다면, 이 단어의 named entity score 는 0.95 가 됩니다.

In [13]:
from collections import Counter

# word count
word_counter = Counter(y_words)

# prediction count
pred_pos = defaultdict(int)
for row in np.where(y_prob >= 0.3)[0]:
    pred_pos[y_words[row]] += 1
pred_pos = {word:pos/word_counter[word] for word, pos in pred_pos.items()}

## 결과 확인하기

Named entity score 가 큰 순서대로 상위 1000 개의 단어를 선택합니다. 그 중 seed words 에 포함된 단어는 출력하지 않습니다. (단어, 빈도수), score 를 확인합니다. 실제로 `엔헤서웨이` 가 띄어쓰기 된 `헤서웨이` 은 261 번 등장했으며, 약 233 번 0.3 보다 큰 probability 를 얻었습니다.

배우 `유해진`은 삼시 세끼 이후 `참바다`라는 별명을 얻게 되었는데, 영화평에서도 이 단어가 배우 이름으로 이용됨을 알 수 있습니다.

In [14]:
for word, prob in sorted(pred_pos.items(), key=lambda x:-x[1])[:1000]:
    if word in seed_words:
        continue
    if word_counter[word] < 50:
        continue
    idx = vocab_to_idx[word]
    count = idx_to_count[idx]
    print('{} ({})\t{:.3f}'.format(word, count, prob))

패틴슨 (286)	0.902
헤서웨이 (261)	0.893
고슬링 (195)	0.882
와저 (188)	0.851
보성이 (190)	0.842
종석 (183)	0.835
ㅋㅋ이민기 (80)	0.825
보성 (411)	0.798
손혜진 (61)	0.787
진구씨 (171)	0.760
내생 (843)	0.758
류해진 (80)	0.750
보영 (636)	0.746
김래원 (3096)	0.736
김영민 (80)	0.725
감우성씨 (98)	0.724
태현 (58)	0.724
에바그린 (555)	0.723
제니퍼로렌스 (50)	0.720
우성이 (271)	0.720
자매간 (82)	0.720
에디레드메인 (66)	0.712
이재훈 (61)	0.705
라이언고슬링 (134)	0.701
레져 (78)	0.692
데이먼 (942)	0.686
보영이누나 (144)	0.685
최다니엘 (239)	0.684
참바다 (81)	0.679
지연누나 (198)	0.677
민희 (61)	0.672
민정 (68)	0.672
하이모어 (124)	0.669
염정아 (133)	0.669
애덤리바인 (108)	0.664
김아중 (2567)	0.663
부녀간 (65)	0.662
지현언니 (56)	0.661
강호 (147)	0.660
기동이 (1341)	0.655
오웬 (84)	0.655
표도르 (99)	0.646
조니뎁 (3071)	0.643
유승룡 (56)	0.643
남녀간 (164)	0.640
탑오빠 (75)	0.640
그냥말 (259)	0.637
원빈 (10765)	0.635
노아 (265)	0.634
한가인 (800)	0.633
동석 (98)	0.633
진격 (182)	0.632
애드워드 (130)	0.628
아중이 (145)	0.628
하울 (2337)	0.627
문채원 (292)	0.627
샤를리즈테론 (93)	0.624
현빈 (3658)	0.620
화담 (50)	0.620
허이재 (204)	0.616
이요원 (124)	0.613
이진욱 (369)	0.612
톰쿠르즈 (116)	0.61

## Named Entity Filter (Feature) 확인하기

우리는 window instance 를 하나의 row 로 만들었기 때문에 prediction probability 가 높은 instance 를 확인하면, 어떤 context 에서 X[0] 가 사람 이름인지를 확인할 수 있습니다.

In [15]:
train_prob = logistic.predict_proba(X_train)
top_instances = np.where(0.7 <= train_prob[:,1])[0]
top_probs = train_prob[top_instances,1]

print(top_instances.shape)
print(top_probs.shape)

(121960,)
(121960,)


121,960 개의 rows 중에는 중복된 것들도 많습니다. 중복된 경우를 정리하여 각 instance 와 count, 그리고 prediction probability 를 정리하는 함수를 만듭니다.

In [16]:
from collections import defaultdict

def get_unique_top_instances(X, sample_idxs, probs):

    # slice samples
    X_samples = X[sample_idxs]
    rows, cols = X_samples.nonzero()

    # find unique instance
    instance_prob = {}
    instance_count = defaultdict(int)
    before_row = None

    def update_dict(features, prob):
        features = sorted(features, key=lambda x:x[0])
        instance = ', '.join(f[1] for f in features)
        instance_prob[instance] = prob
        instance_count[instance] += 1
        return []

    features = [] # temporal variable
    for row, feature_idx in zip(rows, cols):
        # update unique dictionary
        if row != before_row and features:
            features = update_dict(features, probs[row])
        # update temporal variable
        before_row = row
        feature = idx_to_feature(feature_idx, idx_to_vocab, window=2)
        features.append((feature_idx, feature))

    # last elements
    if features:
        update_dict(features, probs[row])

    return instance_prob, instance_count

여진히 unique instance (unique input vectors) 의 개수가 많습니다.

In [17]:
instance_prob, instance_count = get_unique_top_instances(X_train, top_instances, top_probs)
len(instance_prob)

86816

빈도수 기준으로 상위 500 개의 instance 를 출력합니다 (probability, count), instance 입니다.

영화평에서는 배우의 이름들이 반복하여 등장하는 경우들이 많습니다. 그러한 관용구들이 아래와 같은 템플릿으로 학습됩니다.

```
(0.9405, 52)	X[-2] = 황정민, X[-1] = 과, X[1] = 의, X[2] = 케미
(0.9751, 88)	X[-2] = 을, X[-1] = 위한, X[1] = 에, X[2] = 의한
(0.8301, 45)	X[-2] = 솔직, X[-1] = 히, X[1] = 때문, X[2] = 에
```

In [18]:
sorted_instance_count = sorted(instance_count.items(), key=lambda x:-x[1])

for instance, count in sorted_instance_count[:500]:
    prob = instance_prob[instance]
    print('({:.4}, {})\t{}'.format(prob, count, instance))

(0.7002, 1344)	X[1] = 의, X[2] = 연기
(0.9341, 1030)	X[1] = 때문, X[2] = 에
(0.7056, 806)	X[1] = 의, X[2] = 연
(0.9397, 598)	X[1] = 의, X[2] = 연기력
(0.718, 548)	X[1] = 씨, X[2] = 연기
(0.7298, 495)	X[-2] = 관람, X[-1] = 객, X[1] = 연, X[2] = 기가
(0.8522, 490)	X[-2] = 관람, X[-1] = 객, X[1] = 의, X[2] = 연
(0.8162, 423)	X[-2] = 관람, X[-1] = 객, X[1] = 의, X[2] = 연기
(0.991, 358)	X[1] = 의, X[2] = 재
(0.9865, 259)	X[-2] = 관람, X[-1] = 객, X[1] = 의, X[2] = 연기력
(0.7456, 231)	X[1] = 을, X[2] = 위한
(0.835, 218)	X[1] = 의, X[2] = 매력
(0.9951, 196)	X[-2] = 관람, X[-1] = 객, X[1] = 연기, X[2] = 는
(0.8036, 192)	X[-2] = 관람, X[-1] = 객, X[1] = 을, X[2] = 위한
(0.7718, 188)	X[-2] = 관람, X[-1] = 객, X[1] = 때문, X[2] = 에
(0.9054, 174)	X[-2] = 관람, X[-1] = 객, X[1] = 씨, X[2] = 연기
(0.888, 173)	X[-2] = 에, X[-1] = 의한, X[1] = 을, X[2] = 위한
(0.7207, 140)	X[-2] = 관람, X[-1] = 객, X[1] = 연기, X[2] = 너무
(0.7982, 139)	X[1] = 씨, X[2] = 연
(0.9267, 137)	X[-2] = 관람, X[-1] = 객, X[1] = 가, X[2] = 너무
(0.7632, 129)	X[1] = 님, X[2] = 연기
(0.8118, 122)	X[1] = 의, X[2] = 눈빛
(0

우리는 sparse vector representation 형태의 input data 를 만들었기 때문에 logistic regression 을 이용했습니다. 이와 다르게 X[-2], X[-1], X[1], X[2] 의 단어의 word vector 를 concatenation 한다면 neural network 의 input 이 될 수 있습니다. 이러한 방식이 아래의 Stanford 수업의 assignment 내용입니다.

https://nlp.stanford.edu/~socherr/pa4_ner.pdf

또한 word vector 를 이용한다면 Convolutional Neural Network (CNN) 를 이용할 수도 있습니다. 특히나 CNN 은 n-gram features 를 이용하기 때문에 문맥 정보가 더 정확히 잡힐 수 있습니다. 이에 대해서는 다른 튜토리얼에서 다룹니다.